# Companies Political Donation History

Here you can search for the donation history of all Political Action Committees (PACs). Many companies have their own Political Action Committees so you can directly search a company name such as Amazon or Facebook. 

The results for each PAC will show the members of the House and Senate they have donated to, the amounts donated, and also the voting history of these candidates as rated by three groups: the American Civil Libirties Union (ACLU), the National Association for the Advancement of Colored People (NAACP), and League of Conservation Voters (LCV). 

These groups compiled scorecards which rate members of congress votes based on whether the members of congress agreed with the group's position on certain pieces of legislation.
More about each groups scorecard can be found at:
- [ACLU Scorecard](https://www.aclu.org/other/legislative-scorecard-2018)
- [NAACP Scorecard](https://www.naacp.org/report-cards/)
- [LCV Scorecard](https://scorecard.lcv.org/)

All the PAC donation information is filed with the [Federal Election Committee](https://www.fec.gov/data/).

In [4]:
from __future__ import division

# Load Open FEC data
from tqdm.notebook import tqdm
pbar = tqdm(leave=False)

In [ ]:
import pandas as pd

def load_df_from_openFEC(zip_url, zip_file, header_url):
   
    from io import BytesIO  
    from urllib.request import urlopen
    from zipfile import ZipFile
    
    columns = []
    # Load header data
    with urlopen(header_url) as headresp:
        df  = pd.read_csv(BytesIO(headresp.read()), sep=',', header=None)
        columns = df.iloc[0].to_list()
    
    # Load contents
    with urlopen(zip_url) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            df = pd.read_csv(zfile.open(zip_file), sep="|", header=None, low_memory=False, names=columns)
            return df
        
def get_candidate_score():
    num_possible_scores = 3
    scorecard_df['candidate_score'] = \
        sum(scorecard_df['2017_aclu_score','2019_environment_score','2017_naacp_score']) \
        / (num_possible_scores - sum(scorecard_df['2017_aclu_score','2019_environment_score','2017_naacp_score'].isna()) )
    return scorecard_df

pbar.reset(total=4)
pbar.set_description("Loading datafiles")

donations_df = load_df_from_openFEC(
    "https://www.fec.gov/files/bulk-downloads/2020/pas220.zip", "itpas2.txt",
    "https://www.fec.gov/files/bulk-downloads/data_dictionaries/pas2_header_file.csv")
pbar.update()

cmte_df = load_df_from_openFEC(
    "https://www.fec.gov/files/bulk-downloads/2020/cm20.zip", "cm.txt",
    "https://www.fec.gov/files/bulk-downloads/data_dictionaries/cm_header_file.csv")
cmte_df = cmte_df.set_index('CMTE_ID')
pbar.update()

cand_df = load_df_from_openFEC(
    "https://www.fec.gov/files/bulk-downloads/2020/cn20.zip", "cn.txt",
    "https://www.fec.gov/files/bulk-downloads/data_dictionaries/cn_header_file.csv")
cand_df = cand_df.set_index('CAND_ID')
pbar.update()

scorecard_file = './congressional_scorecards/complete_scorecard.csv'
scorecard_df = pd.read_csv(scorecard_file, sep=",", header=0, low_memory=False, index_col=3)
scorecard_df.drop['candidate_score']
get_candidate_score()
pbar.update()
pbar.close()

In [35]:
import ipywidgets as ipw

cb_input = ipw.Combobox(description='', options=[str(v) for v in cmte_df.CMTE_NM.tolist()], placeholder="Start typing here.")
html_output = ipw.HTML()

def on_go(btn):
    cmte_id_sel = cmte_df.index[cmte_df.CMTE_NM == cb_input.value]
    if cmte_id_sel.empty:
        html_output.value = "PAC not found"
        # TODO make popup of company not found
        return
    sc_df = get_scorecard(cmte_id_sel.format()[0])
    display_columns = ['CAND_NAME','CAND_OFFICE_ST','TRANSACTION_AMT','CAND_OFFICE','2017_aclu_score','2019_environment_score','2017_naacp_score']
    html_output.value = sc_df.to_html(header=False,columns=display_columns,index=False)
    #disp_df = sc_df.reindex(columns=['CAND_NAME','TRANSACTION_AMT'])

def on_clear(btn):
    cb_input.value = ''
    html_output.value = ''

def get_scorecard(cmte_id):
    cmte_donations_df = donations_df[donations_df.CMTE_ID == cmte_id] # Select all donations where the CMTE_ID == cmte_id_sel

    cmte_donations_df = cmte_donations_df.groupby('CAND_ID').TRANSACTION_AMT.sum()   # Sum transactions
    cmte_donations_df = pd.DataFrame(cmte_donations_df)
    cmte_donations_df = cmte_donations_df.merge(cand_df, how='left', left_index=True, right_index=True, suffixes=('',''))   # Merge candidate data into result
    cmte_donations_df = cmte_donations_df.merge(scorecard_df,how='left', left_index=True, right_index=True, suffixes=('',''))

    cmte_donations_df = cmte_donations_df.sort_values(by = ['CAND_OFFICE','candidate_score','CAND_OFFICE_ST','CAND_NAME'])
    
    return cmte_donations_df

go_button = ipw.Button(description='Go!', layout=ipw.Layout(width="45px"))
go_button.on_click(on_go)

clear_button = ipw.Button(description='Clear', layout=ipw.Layout(width="90px"))
clear_button.on_click(on_clear)

row1 = ipw.HBox((cb_input, go_button, clear_button))
row2 = html_output
ipw.VBox((row1, row2))


